In [15]:
import numpy as np
import matplotlib as mpl

In [16]:
# Question 1
#: the diameter of the wire (x0), 
#the mean of the diameter of coil(x1)
#and the number of active coils (x2)

In [17]:
arrX = np.array([1, 2, 3])

In [18]:
#implementation of f(x)

In [22]:
def f(x):
    return (x[0] ** 2)*x[1]*(2 + x[2])

In [23]:
print(f(arrX))

10


In [ ]:
#implementation of g1(x)

In [24]:
def g1(x):
    return (1 - (((x[1]**3)*x[2])/71785*(x[0]**4)))

In [21]:
print